In [ ]:
# %pip install -q google-generativeai google-ai-generativelanguage==0.6.15 gradio langchain llama-index pandas openai anthropic langchain_google_genai langchain_community google-api-python-client

In [ ]:
# Bloco 2: Importações Iniciais, Configuração de Logging e Configurações da Aplicação
import os
import logging
import google.generativeai as genai
import gradio as gr
# from google.colab import userdata # Descomente se estiver no Colab e usando secrets

# Configuração do Logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(name)s - %(module)s - %(message)s')
logger = logging.getLogger(__name__)

# Configurações da Aplicação (Externalizadas)
APP_CONFIG = {
    "gemini_model": {
        "model_id": "gemini-2.0-flash",
        "generation_config": {
            "temperature": 0.7,
            "top_p": 0.95,
            "top_k": 40,
            "max_output_tokens": 8192,
        },
        "safety_settings": [
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        ],
        "request_timeout": 120, # Timeout em segundos para a chamada da API (AUMENTADO PARA 120s)
    },
    "rag": {
        "embeddings_model_name": "models/embedding-001",
        "retriever_k": 2
    }
}

print("Bloco 2: Logging e configurações da aplicação (com timeout) definidos.")

In [ ]:
# Bloco 3: Configuração da API Key do Google e Cliente Generative AI
try:
    GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
    # from google.colab import userdata
    # GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

    if GOOGLE_API_KEY:
        os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
        logger.info("GOOGLE_API_KEY carregada com sucesso.")
        genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
        logger.info("Cliente Google Generative AI configurado.")
    else:
        logger.critical("Erro Crítico: A secret 'GOOGLE_API_KEY' ou variável de ambiente não foi encontrada.")
        raise RuntimeError("GOOGLE_API_KEY not found. A execução não pode continuar sem a API Key.")
except Exception as e:
    logger.exception(f"Ocorreu um erro inesperado ao configurar a API Key ou o cliente GenAI: {e}")
    raise
print("Bloco 3: Configuração da API Key e cliente GenAI processada.")

In [ ]:
# Bloco 4: Configuração do Modelo Gemini e Função Auxiliar invocar_gemini
try:
    model_config = APP_CONFIG["gemini_model"]
    model = genai.GenerativeModel(
        model_name=model_config["model_id"],
        generation_config=model_config["generation_config"],
        safety_settings=model_config["safety_settings"]
    )
    logger.info(f"Modelo Gemini '{model_config['model_id']}' inicializado com sucesso.")
except Exception as e:
    logger.exception(f"Erro ao inicializar o modelo Gemini '{model_config['model_id']}': {e}")
    raise

def invocar_gemini(prompt_usuario: str, contexto_sistema: str = "Você é um assistente de pesquisa científica altamente qualificado.") -> str:
    full_prompt = (
        f"{contexto_sistema}\n\n"
        "--- INÍCIO DA TAREFA DO USUÁRIO ---\n"
        f"{prompt_usuario}\n"
        "--- FIM DA TAREFA DO USUÁRIO ---"
    )
    logger.info(f"Invocando Gemini. Contexto: '{contexto_sistema[:50]}...', Prompt (primeiros 100 chars): {prompt_usuario[:100]}...")

    request_options = {"timeout": APP_CONFIG["gemini_model"].get("request_timeout", 60)} # Default 60s, configurado para 120s

    try:
        logger.debug(f"Enviando para Gemini com timeout de {request_options['timeout']}s. Prompt completo (primeiros 200 chars): {full_prompt[:200]}")
        response = model.generate_content(full_prompt, request_options=request_options)
        logger.info("Resposta recebida do Gemini.")

        if response.parts:
            generated_text = "".join(part.text for part in response.parts if hasattr(part, 'text'))
            logger.debug(f"Texto gerado (das partes): {generated_text[:100]}...")
            return generated_text
        elif hasattr(response, 'text') and response.text:
             generated_text = response.text
             logger.debug(f"Texto gerado (do atributo text): {generated_text[:100]}...")
             return generated_text
        else:
            if response.prompt_feedback and response.prompt_feedback.block_reason:
                block_reason_message = getattr(response.prompt_feedback, 'block_reason_message', str(response.prompt_feedback.block_reason))
                logger.warning(f"Conteúdo bloqueado pelo Gemini. Razão: {block_reason_message}")
                return f"Conteúdo bloqueado pela API. Razão: {block_reason_message}"
            if response.candidates and response.candidates[0].content and response.candidates[0].content.parts:
                 generated_text = "".join(part.text for part in response.candidates[0].content.parts if hasattr(part, 'text'))
                 logger.debug(f"Texto gerado (dos candidatos): {generated_text[:100]}...")
                 return generated_text
            logger.warning("Resposta do Gemini vazia ou em formato inesperado. Feedback: %s", response.prompt_feedback)
            return "Resposta vazia ou em formato inesperado do modelo. Verifique os logs e a configuração de segurança."
    except Exception as e:
        logger.exception(f"Erro crítico ao interagir com o modelo Gemini: {e}")
        return f"Erro crítico ao contatar o modelo Gemini: {str(e)}. Verifique os logs do servidor."

print("Bloco 4: Modelo Gemini e função invocar_gemini (com timeout e logging) definidos.")

In [ ]:
# Bloco 5: Definição da Classe Base para Módulos (ModuloSynapseBase)
class ModuloSynapseBase:
    def __init__(self, nome_modulo: str, contexto_sistema_especifico: str, instrucao_principal: str):
        self.nome_modulo = nome_modulo
        self.contexto_sistema_completo = (
            f"Você é o {self.nome_modulo} do Synapse Intellect. "
            f"{contexto_sistema_especifico} "
            "Responda de forma estruturada, seguindo as diretrizes fornecidas. "
            "A entrada do usuário estará claramente delimitada."
        )
        self.instrucao_principal = instrucao_principal

    def construir_prompt_usuario_final(self, entrada_usuario: str) -> str:
        return (
            f"{self.instrucao_principal}\n\n"
            f"ENTRADA DO USUÁRIO PARA ANÁLISE:\n'''\n{entrada_usuario}\n'''\n\n"
            "POR FAVOR, PROCEDA COM A ANÁLISE CONFORME AS DIRETRIZES DO MÓDULO."
        )

    def executar(self, entrada_usuario: str) -> str:
        if not entrada_usuario or not entrada_usuario.strip():
            logger.warning(f"Entrada vazia recebida para o {self.nome_modulo}.")
            return "Por favor, forneça uma entrada válida para o módulo."
        prompt_usuario_final = self.construir_prompt_usuario_final(entrada_usuario)
        logger.info(f"Executando {self.nome_modulo} com entrada (primeiros 100 chars): {entrada_usuario[:100]}...")
        return invocar_gemini(prompt_usuario_final, self.contexto_sistema_completo)

print("Bloco 5: Classe Base ModuloSynapseBase definida.")

In [ ]:
# Bloco 6: Implementação dos Módulos Funcionais

class ModuloInteligenciaEstrategica(ModuloSynapseBase):
    def __init__(self):
        super().__init__(
            nome_modulo="Módulo de Inteligência Estratégica",
            contexto_sistema_especifico="Sua especialidade é identificar tendências, lacunas e oportunidades em pesquisa.",
            instrucao_principal=(
                "Analise o seguinte tema ou problema de pesquisa fornecido.\nSua análise deve incluir:\n"
                "1. Tendências emergentes e consolidadas.\n2. Lacunas de conhecimento.\n"
                "3. Possíveis impactos futuros.\n4. Palavras-chave relevantes.\nSeja prospectivo e estratégico."
            )
        )
def modulo_a_inteligencia_estrategica(e: str): return ModuloInteligenciaEstrategica().executar(e)

class ModuloIdeacaoValidacao(ModuloSynapseBase):
    def __init__(self):
        super().__init__(
            nome_modulo="Módulo de Ideação e Validação",
            contexto_sistema_especifico="Seu foco é gerar e refinar ideias de pesquisa, avaliando originalidade e conexões.",
            instrucao_principal=(
                "Trabalhe com a seguinte ideia ou problema de pesquisa:\nSua tarefa é:\n"
                "1. Gerar 3-5 novas perguntas de pesquisa.\n2. Avaliar originalidade.\n"
                "3. Sugerir 2-3 conexões interdisciplinares.\n4. Apontar desafios.\nFomente criatividade e rigor."
            )
        )
def modulo_b_ideacao_e_validacao(e: str): return ModuloIdeacaoValidacao().executar(e)

class ModuloArquiteturaMetodologica(ModuloSynapseBase):
    def __init__(self):
        super().__init__(
            nome_modulo="Módulo de Arquitetura Metodológica",
            contexto_sistema_especifico="Sua especialidade é desenhar planos de pesquisa robustos.",
            instrucao_principal=(
                "Desenvolva um esboço de plano metodológico para:\nSeu esboço deve considerar:\n"
                "1. Objetivos SMART.\n2. Abordagem metodológica.\n3. Amostragem/corpus.\n"
                "4. Fases da pesquisa.\n5. Instrumentos/técnicas.\nEstrutura clara e lógica."
            )
        )
def modulo_c_arquitetura_metodologica(e: str): return ModuloArquiteturaMetodologica().executar(e)

class ModuloRevisaoCriticaEtica(ModuloSynapseBase):
    def __init__(self):
        super().__init__(
            nome_modulo="Módulo de Revisão Crítica & Ética",
            contexto_sistema_especifico="Atue como revisor experiente: rigor, viabilidade, ética.",
            instrucao_principal=(
                "Analise a seguinte proposta/ideia/rascunho:\nSua análise crítica deve abordar:\n"
                "1. Clareza do problema/objetivos.\n2. Viabilidade.\n3. Potencial de impacto.\n"
                "4. Possíveis vieses e mitigação.\n5. Considerações éticas.\nSeja construtivo e rigoroso."
            )
        )
def modulo_d_revisao_critica_e_etica(e: str): return ModuloRevisaoCriticaEtica().executar(e)

class ModuloEscritaCientifica(ModuloSynapseBase):
    def __init__(self):
        super().__init__(
            nome_modulo="Módulo de Escrita Científica",
            contexto_sistema_especifico="Ajude a aprimorar textos acadêmicos: clareza, estrutura, rigor.",
            instrucao_principal=(
                "Trabalhe com o seguinte texto/solicitação:\nSua tarefa pode ser (adapte):\n"
                "1. Rascunho de seção: melhorias.\n2. Ideia para parágrafo/seção: desenvolvimento.\n"
                "3. Dúvida sobre estilo/normas: orientações gerais.\n4. Pedido de esboço: estrutura típica.\nPriorize objetividade, precisão, concisão."
            )
        )
def modulo_e_escrita_cientifica(e: str): return ModuloEscritaCientifica().executar(e)

class ModuloImpactoOperacoes(ModuloSynapseBase):
    def __init__(self):
        super().__init__(
            nome_modulo="Módulo de Impacto & Operações Científicas",
            contexto_sistema_especifico="Oriente sobre estratégias de publicação, visibilidade e gestão.",
            instrucao_principal=(
                "Forneça orientações estratégicas para:\nSuas orientações devem cobrir:\n"
                "1. Sugestões de periódicos/conferências.\n2. Estratégias para visibilidade/impacto.\n"
                "3. KPIs para monitorar progresso.\n4. Dicas para submissão.\nConselhos práticos."
            )
        )
def modulo_f_impacto_e_operacoes_cientificas(e: str): return ModuloImpactoOperacoes().executar(e)

class ModuloCientometriaAplicada(ModuloSynapseBase):
    def __init__(self):
        super().__init__(
            nome_modulo="Módulo de Cientometria Aplicada",
            contexto_sistema_especifico="Explique e ajude a interpretar métricas científicas (conhecimento geral).",
            instrucao_principal=(
                "Analise a seguinte consulta/dados sobre métricas:\nSua resposta deve:\n"
                "1. Pergunta geral: explique conceito, cálculo, uso, limitações.\n"
                "2. Autor/artigo/revista: métricas relevantes e interpretação.\n"
                "3. Dados numéricos: interpretação contextualizada.\nSem acesso a bases de dados em tempo real."
            )
        )
def modulo_g_cientometria_aplicada(e: str): return ModuloCientometriaAplicada().executar(e)

print("Bloco 6: Módulos funcionais (todos usando Classe Base) definidos.")

In [ ]:
# Bloco 7: Mapeamento dos Módulos para a Interface Gradio
modulos_disponiveis = {
    "A. Inteligência Estratégica": modulo_a_inteligencia_estrategica,
    "B. Ideação e Validação": modulo_b_ideacao_e_validacao,
    "C. Arquitetura Metodológica": modulo_c_arquitetura_metodologica,
    "D. Revisão Crítica & Ética": modulo_d_revisao_critica_e_etica,
    "E. Escrita Científica": modulo_e_escrita_cientifica,
    "F. Impacto & Operações Científicas": modulo_f_impacto_e_operacoes_cientificas,
    "G. Cientometria Aplicada": modulo_g_cientometria_aplicada,
}
nomes_modulos_dropdown = list(modulos_disponiveis.keys())

logger.info("Mapeamento de módulos para Gradio criado.")
print("Bloco 7: Mapeamento de módulos para Gradio criado.")

In [ ]:
# Bloco 8: Esboço Conceitual para Geração Aumentada por Recuperação (RAG) - Comentado
# Esta seção é um placeholder e requer implementação com seus próprios dados e configuração detalhada.
# As bibliotecas LangChain e LlamaIndex foram instaladas no Bloco 1.

# --- INÍCIO DO CÓDIGO RAG DE EXEMPLO (DESCOMENTE E ADAPTE PARA TESTAR) ---

# --- Preparação de Dados de Exemplo (VOCÊ PRECISA CRIAR ESTES ARQUIVOS) ---
# Exemplo: Crie 'doc_exemplo_1.txt' e 'doc_exemplo_2.txt' no seu ambiente.

# rag_retriever = None
# rag_chain = None

# def configurar_rag_simples():
#     global rag_retriever, rag_chain
#     logger.info("Tentando configurar o RAG simples...")
#     try:
#         # Verifique se os arquivos de exemplo existem
#         if not (os.path.exists("doc_exemplo_1.txt") and os.path.exists("doc_exemplo_2.txt")):
#             logger.warning("Arquivos de exemplo para RAG (doc_exemplo_1.txt, doc_exemplo_2.txt) não encontrados. RAG não será configurado.")
#             return

#         from langchain_community.document_loaders import TextLoader
#         from langchain.text_splitter import RecursiveCharacterTextSplitter
#         from langchain_google_genai import GoogleGenerativeAIEmbeddings
#         from langchain_community.vectorstores import FAISS
#         from langchain.chains import RetrievalQA
#         from langchain_google_genai import ChatGoogleGenerativeAI

#         loader1 = TextLoader('doc_exemplo_1.txt', encoding='utf-8')
#         loader2 = TextLoader('doc_exemplo_2.txt', encoding='utf-8')
#         documents = loader1.load() + loader2.load()

#         if not documents:
#             logger.warning("Nenhum documento carregado para o RAG.")
#             return

#         text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
#         docs_split = text_splitter.split_documents(documents)

#         if not docs_split:
#             logger.warning("Nenhum documento dividido para o RAG.")
#             return

#         embeddings = GoogleGenerativeAIEmbeddings(model=APP_CONFIG["rag"]["embeddings_model_name"], task_type="retrieval_document")
#         vector_store = FAISS.from_documents(docs_split, embeddings)
#         rag_retriever = vector_store.as_retriever(search_kwargs={"k": APP_CONFIG["rag"]["retriever_k"]})

#         llm_rag = ChatGoogleGenerativeAI(model=APP_CONFIG["gemini_model"]["model_id"],
#                                          temperature=0.5,
#                                          convert_system_message_to_human=True)

#         rag_chain = RetrievalQA.from_chain_type(
#             llm=llm_rag,
#             chain_type="stuff",
#             retriever=rag_retriever,
#             return_source_documents=True
#         )
#         logger.info("Chain RetrievalQA (RAG) configurado com sucesso usando documentos de exemplo.")

#     except Exception as e:
#         logger.exception(f"Erro ao configurar o exemplo RAG: {e}")
#         rag_chain = None # Garante que não será usado se falhar

# def consultar_rag_com_gemini(query: str) -> str:
#     if rag_chain is None:
#         logger.warning("Sistema RAG (chain RetrievalQA) não configurado. Chamada ignorada.")
#         return "Sistema RAG não está configurado. Verifique a célula de configuração RAG e os arquivos de exemplo."
#     try:
#         logger.info(f"Consultando RAG com query: {query[:100]}...")
#         response = rag_chain.invoke({"query": query})
#         answer = response.get("result", "Não foi possível obter uma resposta do chain RAG.")
#         # source_documents = response.get("source_documents", [])
#         # if source_documents:
#         #     answer += "\n\n--- Documentos Fonte Recuperados (RAG) ---"
#         #     for i, doc in enumerate(source_documents):
#         #         answer += f"\nFonte {i+1}: {doc.page_content[:100]}..."
#         return answer
#     except Exception as e:
#         logger.exception(f"Erro durante a consulta RAG: {e}")
#         return f"Erro durante a consulta RAG: {str(e)}"

# Descomente a linha abaixo para tentar configurar o RAG ao iniciar (requer arquivos de exemplo)
# configurar_rag_simples()

# --- FIM DO CÓDIGO RAG DE EXEMPLO ---

logger.info("Esboço conceitual RAG definido (atualmente comentado).")
print("Bloco 8: Esboço RAG (comentado) definido.")

In [ ]:
# Bloco 9: Função da Interface Gradio (interface_synapse_intellect)
def interface_synapse_intellect(reflexao_inicial, modulo_selecionado, texto_para_modulo, usar_rag_checkbox):
    # Log inicial da requisição
    log_prefix = "Interface Gradio:"
    logger.info(f"{log_prefix} Módulo: '{modulo_selecionado}', RAG: {usar_rag_checkbox}")
    logger.debug(f"{log_prefix} Reflexão: {str(reflexao_inicial)[:100]}...")
    logger.debug(f"{log_prefix} Entrada: {str(texto_para_modulo)[:100]}...")

    if not modulo_selecionado:
        logger.warning(f"{log_prefix} Nenhum módulo selecionado.")
        return "Por favor, selecione um módulo.", "Nenhum módulo foi selecionado."
    if not texto_para_modulo or not str(texto_para_modulo).strip():
        logger.warning(f"{log_prefix} Entrada vazia para '{modulo_selecionado}'.")
        return "Por favor, forneça a entrada específica para o módulo.", f"A entrada para '{modulo_selecionado}' está vazia."

    entrada_completa_para_modulo = str(texto_para_modulo)
    if reflexao_inicial and str(reflexao_inicial).strip():
        entrada_completa_para_modulo = (
            f"Contexto/Reflexão Inicial do Pesquisador:\n'''\n{str(reflexao_inicial)}\n'''\n\n"
            f"Tarefa Específica para o Módulo '{modulo_selecionado}':\n'''\n{str(texto_para_modulo)}\n'''"
        )
        logger.info(f"{log_prefix} Reflexão inicial combinada com a entrada do módulo.")

    resultado_modulo = "Processando..." # Placeholder inicial
    debrief_base_text = f"Synapse Intellect (Módulo: {modulo_selecionado}) processou sua solicitação."

    # A funcionalidade RAG permanece como esboço e não será ativada.
    if usar_rag_checkbox:
        logger.warning(f"{log_prefix} RAG solicitado, mas não está ativo/implementado.")
        debrief_base_text += " (RAG solicitado mas não ativo)."
        # Poderia adicionar um aviso ao resultado_modulo aqui se quisesse.

    funcao_modulo = modulos_disponiveis.get(modulo_selecionado)
    if funcao_modulo:
        logger.info(f"{log_prefix} Chamando função para o módulo '{modulo_selecionado}'.")
        resultado_modulo = funcao_modulo(entrada_completa_para_modulo) # Chamada direta
        logger.info(f"{log_prefix} Resultado recebido do módulo '{modulo_selecionado}'.")
        debrief_base_text += " (Resposta gerada pelo modelo Gemini)."
    else:
        logger.error(f"{log_prefix} Módulo '{modulo_selecionado}' não encontrado no mapeamento.")
        resultado_modulo = f"Erro: Módulo '{modulo_selecionado}' não encontrado."
        debrief_base_text = "Erro crítico: Módulo não implementado."

    # Debrief gerado pelo LLM
    prompt_debrief_llm = (
        f"O usuário interagiu com o '{modulo_selecionado}'. "
        f"Descreva brevemente (1-2 sentenças) a estratégia ou foco principal deste módulo. Não resuma a resposta ao usuário."
    )
    contexto_debrief_llm = "Você é um assistente IA que descreve a função de outros módulos IA de forma concisa."
    debrief_gerado_pelo_llm = invocar_gemini(prompt_debrief_llm, contexto_debrief_llm)

    final_debrief_text = f"{debrief_base_text}\n\nEstratégia do Módulo (sugerida pela IA): {debrief_gerado_pelo_llm}"
    return resultado_modulo, final_debrief_text

print("Bloco 9: Função da interface Gradio definida.")

In [ ]:
# Bloco 10: Criação da Interface Gradio (com CSS Customizado)

custom_css = """
body, .gradio-container {
    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Open Sans', 'Helvetica Neue', sans-serif;
    background-color: #f0f2f5; /* Um cinza muito claro, estilo Apple */
    color: #1d1d1f; /* Texto principal escuro */
}
.gr-panel { /* Usado para agrupar colunas/componentes */
    background-color: #ffffff !important;
    border: 1px solid #d2d2d7 !important; /* Borda sutil */
    border-radius: 12px !important;
    box-shadow: 0 1px 3px rgba(0,0,0,0.04), 0 1px 2px rgba(0,0,0,0.08) !important; /* Sombra mais sutil */
    padding: 24px !important;
}
.gr-button {
    background-color: #007aff !important; /* Azul Apple */
    color: white !important;
    border-radius: 8px !important;
    font-weight: 500 !important;
    border: none !important;
    padding: 10px 18px !important;
    box-shadow: 0 1px 2px rgba(0,0,0,0.05) !important;
}
.gr-button:hover {
    background-color: #0056b3 !important; /* Azul mais escuro no hover */
}
.gr-input input, .gr-textbox textarea, .gr-dropdown select, .gr-checkboxgroup input {
    background-color: #f8f8fa !important; /* Fundo do campo de entrada um pouco mais claro */
    border-radius: 8px !important;
    border: 1px solid #c6c6c8 !important; /* Borda mais sutil */
    color: #1d1d1f !important;
    padding: 10px !important;
}
.gr-dropdown select { padding-right: 30px !important; } /* Espaço para a seta */

.gr-markdown h1 { font-size: 28px; font-weight: 600; color: #1d1d1f; margin-bottom: 12px; }
.gr-markdown h3 { font-size: 18px; font-weight: 600; color: #1d1d1f; margin-top: 20px; margin-bottom: 8px; }
.gr-markdown p, .gr-markdown li { color: #333333; line-height: 1.6; }

label span, .gr-checkbox label span { /* Texto dos labels */
    color: #555 !important;
    font-weight: 500 !important;
    font-size: 14px !important;
}
.gr-info { /* Texto de informação abaixo dos componentes */
    color: #6e6e73 !important;
    font-size: 12px !important;
}
#resultado_modulo_md { /* Saída principal Markdown */
    background-color: #f8f9fa;
    padding: 18px;
    border-radius: 8px;
    border: 1px solid #e9ecef;
    min-height: 250px; /* Altura mínima para a área de resultado */
}
#output_debrief_modulo textarea { /* Textbox do Debrief */
    background-color: #e9ecef !important; /* Fundo levemente diferente para o debrief */
    font-size: 13px !important;
    line-height: 1.5 !important;
}
/* Remover borda padrão do Gradio container se o tema Soft adicionar uma */
.gradio-container { border: none !important; box-shadow: none !important; }
"""
try:
    with gr.Blocks(theme=gr.themes.Default(text_size=gr.themes.sizes.text_md), css=custom_css, title="Synapse Intellect™ v1.3") as app_synapse: # Mudei para Default para ter mais controle com CSS
        gr.Markdown(
            """
            # 🧠 Synapse Intellect™
            ### v1.3 - Seu Copiloto de IA para Pesquisa Científica
            """
        )
        with gr.Row(variant="panel"): # Usando variant="panel" para aplicar o estilo .gr-panel
            with gr.Column(scale=1):
                gr.Markdown("### Fase 0: Reflexão Inicial (Opcional)")
                input_reflexao = gr.Textbox(
                    label="Descreva seu problema de pesquisa ou ideia geral aqui",
                    lines=3, # Reduzido para interface mais compacta
                    placeholder="Ex: Dificuldades na adesão ao tratamento..."
                )
                gr.Markdown("### ⚙️ Módulo e Entrada")
                input_modulo_selecionado = gr.Dropdown(
                    choices=nomes_modulos_dropdown,
                    label="Selecione o Módulo Funcional",
                    info="Escolha a função que deseja utilizar."
                )
                input_texto_para_modulo = gr.Textbox(
                    label="Entrada Específica para o Módulo",
                    lines=6, # Reduzido
                    placeholder="Forneça detalhes, perguntas, temas..."
                )
                input_usar_rag = gr.Checkbox(
                    label="Usar RAG (Funcionalidade em Esboço - NÃO ATIVA)",
                    value=False,
                    interactive=False # Mantido desabilitado
                )
                botao_processar = gr.Button("🚀 Processar com Synapse Intellect")

            with gr.Column(scale=2):
                gr.Markdown("### 💡 Resultados Gerados")
                output_resultado_modulo = gr.Markdown(elem_id="resultado_modulo_md", label="Resultado do Módulo")
                output_debrief_modulo = gr.Textbox(
                    elem_id="output_debrief_modulo",
                    label="Synapse Strategy Debrief™",
                    lines=6, # Reduzido
                    interactive=False,
                    show_copy_button=True
                )
        gr.Markdown(
            """
            ---
            <p style="text-align:center; font-size:12px; color:#888;">
            Protótipo v1.3. Valide criticamente as respostas. RAG não ativo.
            </p>
            """
        )
    logger.info("Interface Gradio v1.3 definida com CSS customizado.")
    print("Bloco 10: Interface Gradio v1.3 definida.")
except Exception as e:
    logger.exception("Erro ao definir a interface Gradio v1.3.")
    raise

In [ ]:
# Bloco 11: Lançamento da Interface Gradio (Execução Principal)

# Certifique-se que 'app_synapse' foi definido no Bloco 10
if 'app_synapse' in locals():
    logger.info("Iniciando a interface Gradio do Synapse Intellect™ v1.3...")
    # A linha abaixo é a que efetivamente inicia o Gradio em um notebook
    app_synapse.launch(share=True, debug=True, inline=False)
    logger.info("Interface Gradio iniciada. Verifique o link público acima.")
else:
    logger.error("O objeto 'app_synapse' (interface Gradio) não foi definido. Verifique erros nos blocos anteriores, especialmente o Bloco 10.")

print("Bloco 11: Tentativa de lançamento da interface Gradio concluída.")
print("Se a interface iniciou, você verá um link '.gradio.live' na saída acima desta mensagem.")
print("Se não houver link ou houver erros, revise as saídas das células anteriores e os logs.")

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://aec6e9436b66f065de.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:__main__:Erro crítico ao interagir com o modelo Gemini: HTTPConnectionPool(host='localhost', port=41941): Read timed out. (read timeout=120.0)
Traceback (most recent call last):
  File "<ipython-input-31-d33493dd05d3>", line 27, in invocar_gemini
    response = model.generate_content(full_prompt, request_options=request_options)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/generativeai/generative_models.py", line 331, in generate_content
    response = self._client.generate_content(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 868, in generate_content
    # - It may require specifying regional endpoints when creating the service
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google